# **Best submission**

# Libraries

In [1]:
!pip install dataprep

In [1]:
import dataprep
from dataprep import eda
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.impute import SimpleImputer
from sklearn import preprocessing

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load datasets

In [1]:
# Get the train data
trainFile = "../input/titanic/train.csv"
trainDataset = pd.read_csv(trainFile)
print(trainDataset.shape)
# Get the test data
testFile = "../input/titanic/test.csv"
testDataset = pd.read_csv(testFile)
print(testDataset.shape)

# **Analys**

In [1]:
eda.create_report(trainDataset, title="titanic report")

In [1]:
trainDataset.describe()

In [1]:
trainDataset.info()

# Data preparation

In [1]:
train_y=trainDataset['Survived']
del trainDataset['Survived']

del testDataset['Name']
del trainDataset['Name']

del testDataset['Ticket']
del trainDataset['Ticket']

In [1]:
data = [trainDataset,testDataset]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'travelled_alone'] = 0
    dataset.loc[dataset['relatives'] == 0, 'travelled_alone'] = 1
trainDataset

In [1]:
data=pd.concat([trainDataset,testDataset])
data

In [1]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(data['Sex'])
testDataset['Sex']=label_encoder.transform(testDataset['Sex'])
trainDataset['Sex']=label_encoder.transform(trainDataset['Sex'])

label_encoder.fit(data['Embarked'].astype(str))
testDataset['Embarked']=label_encoder.transform(testDataset['Embarked'].astype(str))
trainDataset['Embarked']=label_encoder.transform(trainDataset['Embarked'].astype(str))

label_encoder.fit(data['Cabin'].astype(str))
testDataset['Cabin']=label_encoder.transform(testDataset['Cabin'].astype(str))
trainDataset['Cabin']=label_encoder.transform(trainDataset['Cabin'].astype(str))

In [1]:
data=pd.concat([trainDataset,testDataset])
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(data)
data_test=imputer.transform(testDataset)
data_train=imputer.transform(trainDataset)

In [1]:
data=np.concatenate((trainDataset,testDataset))
data

# Train

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
sc = StandardScaler() 
data = sc.fit_transform(data)
data_test = sc.transform(data_test)
data_train = sc.transform(data_train)

In [1]:
parameters = {'min_samples_leaf': [ 4,5, 6,7],'max_depth': [7,8,9,10],'max_features': [5,6, 7,8]}
rfc = RandomForestClassifier(n_estimators=200, random_state=42)
gcv = GridSearchCV(rfc, parameters,n_jobs=None, cv=5, verbose=3,scoring='roc_auc', return_train_score=True)
gcv.fit(data_train, train_y)
gcv.score(data_train, train_y)

In [1]:
model=gcv.best_estimator_
model

# Predict

In [1]:
test_y = pd.DataFrame(model.predict(data_test),columns = ['Survived'])
final_df = pd.concat([testDataset['PassengerId'],test_y],axis=1)

In [1]:
# Saving the final dataframe as a .csv file.
final_df.to_csv('submission.csv', index = False)

![image.png](attachment:874bf8ad-c837-4e6a-880a-7c578b15790c.png)